#### Before you run solve this make sure you have created a new conda environment.
run the following commands in your commandline
<pre><code>
conda create -n deepmoji anaconda
source activate deepmoji
## add to jupyter notebook list
python -m ipykernel install --user --name deepmoji --display-name "Python (deepmoji)"</code></pre>

### Or preferably load this notebook into [google colab](https://colab.research.google.com)   


## Transfer learning and bias detection
Today we are gonna practice adopting pretrained language models to power our classifiers. 
Using a pretrained model as input, both means a potentially huge gain in performance, but also a potentially problematic introduction of bias. 

Since you are not controlling the population / dataset from which your model learns it is hard to guarantee that the models do not come with certain biases builtin. 

As the pretrained models come "free", you should instead spent ressources on investigating and potentially eliminating biases (bias correction). Today you will practice investigating the biases. 

We will do this using two datasets: 
1. From the paper:  "Examining Gender and Race Bias in Two Hundred Sentiment Analysis Systems" by Kiritchenko & Mohammad 2018:. [data](https://saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip)

2. Kaggle Toxicity Classification: https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data
Follow the url. Sign in and download the zip file.








In [1]:
## Download the equity evaluation corpus
import requests
response = requests.get('https://saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip')

with open('Equity-Evaluation-Corpus.zip','wb') as f:
    f.write(response.content)

import zipfile
zip_ref = zipfile.ZipFile('Equity-Evaluation-Corpus.zip', 'r')
directory_to_extract_to = 'bias_dataset'

import os
if not os.path.isdir(directory_to_extract_to):
    os.mkdir(directory_to_extract_to)
zip_ref.extractall(directory_to_extract_to)
zip_ref.close()

In [1]:
import pandas as pd
directory_to_extract_to = 'bias_dataset'
bias_df = pd.read_csv(directory_to_extract_to+'/Equity-Evaluation-Corpus.csv')

In [131]:

bias_df.head()

,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed


The dataset contains short sentences expressing a simple sentiment, but with changing characters connotating different genders and ethnicities. This allows you to test your classifier in relation to these biases.

Today we will test two types of classifiers.

- Baseline classifier trained yourself on a given dataset:
    - pick either fasttext.
    - or the NBLOG (Naive Bayes features feed into a Logistic Regression)
    
- And the Deepmoji classifier.


## Setting up the DeepMoji encoder
First we shall see what biases the [DeepMoji](https://arxiv.org/pdf/1708.00524.pdf) encoder has out of the box.

In this way we get to practice loading and interacting with a pretrained model.

DeepMoji was originally conceived using [Keras](https://github.com/bfelbo/DeepMoji), but since you are use to PyTorch we shall use the [TorchMoji](https://github.com/huggingface/torchMoji) implementation.

Loading it is straightforward using git.

In [ ]:
## clone the repository
! git clone https://github.com/huggingface/torchMoji.git

In [ ]:
## download the pretrained model's weights using their script
! python scripts/download_weights.py

In [ ]:
# navigate to the torchmoji folder
import os
os.chdir('TorchMoji')
## install dependencies
! pip install -e .


If already downloaded elsewhere add the deepmoji directory to the sys.path so python can import it automatically

In [132]:
# add to sys.path
import sys
base_path = '' # change if you have downloaded folder elsewhere.
#base_path = '/mnt/b0c8e396-e5ba-4614-be6f-146c4c861ab3/torchMoji/' ## path to the torchmoji directory
#sys.path.insert(0, base_path)


In [140]:
## Load model and tokenizer
from torchmoji.sentence_tokenizer import SentenceTokenizer
# load the deepmoji encoder that transforms text to emojies.
from torchmoji.model_def import torchmoji_emojis
from torchmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH
import json,csv, numpy as np
import warnings; warnings.simplefilter('ignore')


## set the max context length
max_token = 30 ## This will not work for longer texts,
################# here you should consider splitting each text into smaller segments.

# Load vocab (i.e. the index of each word in the vector representation)
with open(VOCAB_PATH, 'r') as f:
    vocabulary = json.load(f)

# initialize tokenizer
sentence_tokenizer = SentenceTokenizer(vocabulary, max_token)
# load model
model = torchmoji_emojis(PRETRAINED_PATH)

The model outputs a vector of length 64 representing the probability of 64 emojiies.

We can find the index of the emojies with descriptions in the data folder.


In [137]:
with open(base_path+'data/emoji_codes.json') as f:
    emoji_desc = json.load(f)
list(emoji_desc.items())[0:10]

[('0', ':joy:'),
 ('1', ':unamused:'),
 ('2', ':weary:'),
 ('3', ':sob:'),
 ('4', ':heart_eyes:'),
 ('5', ':pensive:'),
 ('6', ':ok_hand:'),
 ('7', ':blush:'),
 ('8', ':heart:'),
 ('9', ':smirk:')]

We know use this index and the emoji package to translate the index to emojiies.

In [139]:
import emoji
def translate_emoji(emoji_descr):
    if emoji_descr in emoji.unicode_codes.EMOJI_ALIAS_UNICODE:
        return emoji.unicode_codes.EMOJI_ALIAS_UNICODE[emoji_descr]
    if emoji_descr in emoji.unicode_codes.EMOJI_UNICODE:
        return emoji.unicode_codes.EMOJI_UNICODE[emoji_descr]
    return emoji_descr
to_emoji = [translate_emoji(desc) for i,desc in sorted(emoji_desc.items(),key=lambda x: int(x[0]))]
to_emoji_desc = [desc for i,desc in sorted(emoji_desc.items(),key=lambda x: int(x[0]))]

## index 
to_emoji[0],to_emoji_desc[0]

('😂', ':joy:')

We are now ready to encode the text as emojis

**note we are not using it for transfer learning** but simple as a pretrained classifier.


### Exercise 13.1.1
Use the sentence_tokenizer defined above to tokenize the documents.

see example in the torchmoji examples [e.g.](https://github.com/huggingface/torchMoji/blob/master/examples/encode_texts.py) folder for help.

Inspect the tokenized documents to see the format. Try to convert them back using <code>vocabulary</code> variable defined earlier.

**- Hint this means reversing the vocabulary dictionary.**


In [ ]:
# [Answer to ex. 13.1.1. here]

### Exercise 13.1.2
Encode the tokenized sentences and wrap it in a function.
- Hint: Do a forward pass of the model on the tokenized data.
    - check [here](https://github.com/huggingface/torchMoji/blob/master/examples/encode_texts.py) for help 

For larger datasets and with longer sentences encoding is problematic if not done in batches. 

Write a for loop that takes only 256 tokenized documents at a time and concatenate them to a dataframe in the end.

Use the <code>to_emoji</code> list as columns


In [ ]:
# [Answer to ex 13.1.2 here]

## Ex. 13.1.3
- Join the output of Deepmoji with the bias dataframe columns (Race, Gender and Emotion)
    - Make sure Race count and Gender counts are equal after join.

Investigate if there are significant differences in relation to **Race** (Race column).

- See which types of emojies are most different.


- Make a dictionary mapping emojiis to different classes, either happy, sad, angry etc.
- HINT: Fastests way is to loop through the list of emojiies <code>to_emoji</code> and use the <code>input()</code> to input the class.
        
    

In [ ]:
# [Answer to ex. 13.1.3 here]

### Ex.13.1.4 - See which emotions are most biased

- Groupby Emotion and Race and calculate absolute difference in emoji encoding. 
    - hint: first groupby emotion and race, calculate mean, then diff, then abs and then sum.


In [ ]:
# [Answer to ex. 13.1.4 here]

## Hostility and Minority Dataset (Kaggle Toxicity Classification)
**Context**
All outcome and minority variables are crowdsourced annotations and variables are expressed and percentage of annotators marking the category. This means that to create categorical outcomes we should apply a cutoff. The dataset provider suggests 0.5. 

**Ex.13.2.1:** 
- Define a variable <code>minority_cols</code> as a list of column names of the minorities.
- Define a variable <code>outcome_cols</code> as a list of column names of the minorities.
- Create a categorical version of each variable in the outcome cols and minority cols.


**Ex. 13.2.2:** The dataset is fairly large so subsampling will be a good idea (e.g. 25000 samples) where minorities are upsampled. Do a subsample of 1000 for each minority including a none category. 

**Ex 13.2.3:** Train one of the baseline classifiers (see lecture 13 for bow based and fasttext) on the hostility and minority dataset.

**Ex.13.2.4:** Investigate biases in relation to minority groups.


In [ ]:
# [Answers to ex 13.2.x here]

## Bonus 
So far we have not done any real transfer learning. For this exercise you should visit some of the major models and investigate how to adopt the model to your own dataset.

BERT - [COLAB Example](https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)

DeepMoji:
Try the deepmoji finetuning example [here](https://colab.research.google.com/drive/1IsV5a_tr2c5OVdKnX_PyGjoRWW8DUG0S)
    - HINT: Inspect the load_benchmark data to see how to make your own dataset conform.

In [ ]:
# [Answer to bonus question here]